# Random Forest Classifier

In [4]:
import os
import pickle
import random
import re
import scipy
import torch
import wandb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from label_processor import LabelProcessorSimplified

from lime import lime_text
from lime.lime_text import LimeTextExplainer

from sklearn import metrics
from sklearn.base import clone
from sklearn.compose import make_column_transformer
from sklearn.decomposition import  PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.pipeline import make_pipeline
from sklearn.utils.class_weight import compute_class_weight

from tqdm.auto import tqdm, trange

In [5]:
save_directory = "saved/tfidf/"

In [6]:
def intersection_df(df1, df2, col1=None, col2=None):
    col1 = df1.index if col1 is None else df1[col1]
    col2 = df2.index if col2 is None else df2[col2]
    assert col1.name == col2.name
    ids = set(col1)
    ids.intersection_update(col2)
    df1 = df1[col1.isin(ids)]
    df2 = df2[col2.isin(ids)]
    return df1, df2

def predict_decision_threshold(classifier, X_train, X_test, y_train, y_test):
    y_pred_prob_train = classifier.predict_proba(X_train)[:,1]
    decision_threshold = np.quantile(y_pred_prob_train, 1 - y_train.mean())
    y_pred_prob = classifier.predict_proba(X_test)[:,1]
    y_pred = y_pred_prob > decision_threshold
    return y_pred, y_pred_prob


def evaluate(classifier, X_train, X_test, y_train, y_test):
    y_pred, y_pred_prob = predict_decision_threshold(classifier, X_train, X_test, y_train, y_test)
    
    m = {
        "precision": metrics.precision_score(y_test, y_pred),
        "recall" : metrics.recall_score(y_test, y_pred),
        "f1" : metrics.f1_score(y_test, y_pred),
        "roc_auc" : metrics.roc_auc_score(y_test, y_pred_prob),
        "average_precision": metrics.average_precision_score(y_test, y_pred_prob),
        "matthews": metrics.matthews_corrcoef(y_test,y_pred),
        "spearman":scipy.stats.spearmanr(y_pred_prob, y_test)[0]
    }
    return m

def prepend_keys(d, prepend):
    return {prepend + k:v for k,v in d.items()}

def dropna_both(ds, l):
    indxs = l.dropna().index
    return ds[indxs],  l[indxs]

### Do TF-IDF

In [4]:
## gets message dataset
messages = pd.read_pickle("saved/selected_messages.pickle")

In [5]:
## gets texter, counselor and bot messages by conversation ID
conversations_tfidf = messages.replace('bot', np.nan).dropna().groupby('conversation_id').message.agg(" MESSAGESEP ".join)
conversations_tfidf = conversations_tfidf.fillna("")

In [8]:
messages = pd.read_pickle("saved/selected_messages.pickle")
texter_messages = messages[messages.interaction == 'texter'].groupby('conversation_id').message.agg(" MESSAGESEP ".join)
counselor_messages = messages[messages.interaction == 'counselor'].groupby('conversation_id').message.agg(" MESSAGESEP ".join)
bot_messages = messages[messages.interaction == 'bot'].groupby('conversation_id').message.agg(" MESSAGESEP ".join)
conversations_tfidf = pd.DataFrame({"texter": texter_messages,"counselor":counselor_messages, "bot": bot_messages})
conversations_tfidf = conversations_tfidf.fillna("")

In [9]:
## saving the train conversations for lime explanations
train_ids = torch.load(save_directory+"/train_convos.torch")
train_convos = conversations_tfidf.loc[train_ids]
pickle.dump(train_convos, open(save_directory+"train_set_messages.pickle", "wb"))

OSError: [Errno 28] No space left on device

In [13]:
## saving the test conversations for lime explanations
test_ids = torch.load(save_directory+"/test_convos.torch")
test_convos = conversations_tfidf.loc[test_ids]
pickle.dump(test_convos, open(save_directory+"test_set_messages.pickle", "wb"))

In [10]:
del messages

In [11]:
## TFIDF model
tfidf_transformer = make_pipeline( 
#     TfidfVectorizer(min_df=0.01, max_df=0.4),
    make_column_transformer(
        (TfidfVectorizer(min_df=0.01,max_df=0.4),"texter"),
        (TfidfVectorizer(min_df=0.01,max_df=0.4),"counselor"),
        (TfidfVectorizer(min_df=0.05,max_df=0.4),"bot"),
    ),
    TruncatedSVD(1000)
)

In [12]:
## Fitting TF-IDF
print("Fitting TF-IDF")
tfidf_features = tfidf_transformer.fit_transform(conversations_tfidf)

Fitting TF-IDF


In [13]:
## Saving the TF-IDF features and conversation IDs(?)
pickle.dump(tfidf_features, open(save_directory + "dataset_tfidf.pickle","wb"))
conversations_tfidf.index.to_series().to_pickle(save_directory + "conversation_ids.pickle")

In [11]:
pickle.dump(tfidf_transformer, open(save_directory+"tfidf_transformer.pickle","wb"))

## Attach Labels

In [4]:
## get datasets
dataset_tfidf = pickle.load(open(save_directory+"dataset_tfidf.pickle","rb"))
conversation_ids = pd.read_pickle(save_directory+"conversation_ids.pickle")

In [5]:
## get ids
train_ids = torch.load(save_directory+"/train_convos.torch")
test_ids = torch.load(save_directory+"/test_convos.torch")

In [6]:
## maps numerical position in the dataset to the ids so they can be used to extract corresponding data from arrays
conversation_ids_to_idxs = pd.Series(np.arange(len(conversation_ids)), index=conversation_ids.values)
conversation_idxs_to_ids = pd.Series(conversation_ids.values, index=np.arange(len(conversation_ids)))

In [7]:
## gets labelizer df 
labelizer = torch.load("saved/labelizer.torch")
l_df = labelizer.df

In [8]:
## saves the index 
l_index = l_df.index
# with open(save_directory+'/l_index.pickle', 'wb') as f:
#     pickle.dump(l_index, f)

In [9]:
## gets numeric (not hashed) indexes, for retrieval of the feature data in the next step
# l_index = pickle.load(open(save_directory+"/l_index.pickle", "rb"))
train_indxs = conversation_ids_to_idxs.loc[sorted(train_ids.intersection(conversation_ids).intersection(l_index))].values
test_indxs = conversation_ids_to_idxs.loc[sorted(test_ids.intersection(conversation_ids).intersection(l_index))].values

In [10]:
## gets tf-idf feature data for train and test sets 
ds_train = dataset_tfidf[train_indxs]
ds_test = dataset_tfidf[test_indxs]

## train and test conversation IDs
train_convo_ids = conversation_idxs_to_ids.loc[sorted(train_indxs)].values
test_convo_ids = conversation_idxs_to_ids.loc[sorted(test_indxs)].values

In [11]:
## gets the labels of the train and test sets 
l_train = l_df.loc[train_convo_ids].copy()
l_test = l_df.loc[test_convo_ids].copy()

l_train.set_index(np.arange(len(l_train)), inplace=True)
l_test.set_index(np.arange(len(l_test)), inplace=True)

In [12]:
## save data 
## for some reason, can't torch.save ds_train

with open(save_directory+'ds_train.pickle', 'wb') as f:
    pickle.dump(ds_train, f)
with open(save_directory+'ds_test.pickle', 'wb') as f:
    pickle.dump(ds_test, f)

In [13]:
## save labels 
torch.save(l_train, save_directory+"l_train.torch")
torch.save(l_test, save_directory+"l_test.torch")

# Train TF-IDF Model

In [7]:
## loading dataset and labels for the random forest classifier
ds_train = pickle.load(open(save_directory+"/ds_train.pickle","rb"))
ds_test = pickle.load(open(save_directory+"/ds_test.pickle", "rb"))
l_train = torch.load(save_directory+"l_train.torch")
l_test = torch.load(save_directory+"l_test.torch")

In [8]:
l_train = l_train.rename(columns={"69>13 or younger" : "69_13 or younger",
                                  "69>14-24" : "69_14-24",
                                  "69>24 or younger" : "69_24 or younger"})
l_test = l_test.rename(columns={"69>13 or younger" : "69_13 or younger",
                                "69>14-24" : "69_14-24",
                                "69>24 or younger" : "69_24 or younger"})

In [9]:
def fit_with_cv(X, y, X_cv, y_cv, model, min_estimators, max_estimators, step_size, patience, log_wandb=False):
    if log_wandb:
        import wandb

    best_score = None
    no_improvement_steps = 0
    for i in trange(min_estimators, max_estimators + 1, step_size):
        if no_improvement_steps > patience:
            return
        model.n_estimators = i
        model.fit(X, y)
        prob_cv = model.predict_proba(X_cv)
        score = scipy.stats.spearmanr(y_cv, prob_cv[:, 1])
        if log_wandb:
            wandb.log({"score": score, "n_estimators": model.n_estimators})
        if best_score is None or score > best_score:
            no_improvement_steps = 0
            best_score = score
        else:
            no_improvement_steps += 1

## DeepHelp Impl. 

In [10]:
config={"min_estimators":5,
"max_estimators":5000,
"max_depth":5,
"class_weight":"balanced",
"step_size":5,
"patience":3}

## list of labels to build model for
for label in tqdm([
    "18_anxiety", 
    "19_desire_x",
    "19_intent_x",
    "19_capability_x",
    "19_timeframe_x",
#     "69_13 or younger",
#     "69_14-24",
#     "69_24 or younger",
    "18_substance", 
    "18_depressed", 
    "18_self_harm",
#     "18_suicide",
    "64_Yes",
# #     "65>1 (slightly helpful)",
# #     "65>2",
# #     "65>3",
# #     "65>4",
# #     "241_I see a therapist or doctor in person",
#     "74_Heterosexual or Straight",
#     "75_White",
#     "73_Male",
# #     "33_good"
]):

    wandb.init(project="tfidf_baseline",reinit=True,
               config={
                   **config,
                "label":label
               },tags=['tfidf'])
    
    min_estimators = wandb.config.min_estimators
    max_estimators = wandb.config.max_estimators
    max_depth = wandb.config.max_depth
    class_weight = wandb.config.class_weight
    step_size = wandb.config.step_size
    patience = wandb.config.patience

    X, y = dropna_both(ds_train, l_train[label].astype('float'))
    X_test, y_test = dropna_both(ds_test, l_test[label].astype('float'))

    print(X.shape, y.shape)
    print(X_test.shape, y_test.shape)

    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.1)

    u = np.unique(y_train)
    class_weight = {i:j for i,j in zip(u,compute_class_weight('balanced',u, y_train))}
    
    classifier = RandomForestClassifier(
        n_estimators = min_estimators,
        max_depth = max_depth,
        class_weight = class_weight,
        warm_start=True,
        n_jobs=2)
    
    fit_with_cv(X_train,y_train, X_val, y_val,
                model=classifier, 
                min_estimators = min_estimators, 
                max_estimators = max_estimators, 
                step_size = step_size, 
                patience = patience, 
                log_wandb=True)

    wandb.log({
        **prepend_keys(evaluate(classifier, X_train, X_val, y_train, y_val), "val "),
        **prepend_keys(evaluate(classifier, X_train, X_test, y_train, y_test), "test ")
    })
    print(label)
    print(pd.Series(evaluate(classifier, X_train, X_test, y_train, y_test)))
    wandb.join()

#     with open(f"models/rf/{label}_model.pickle", 'wb') as f:
#         pickle.dump(classifier, f)       
        

wandb: Currently logged in as: kiatann (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


(341955, 1000) (341955,)
(17997, 1000) (17997,)


/data-imperial/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=45955     0.0
99894     0.0
233564    1.0
121034    0.0
90967     1.0
         ... 
176932    1.0
57622     0.0
132151    0.0
168208    0.0
340571    0.0
Name: 18_anxiety, Length: 307759, dtype: float64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



18_anxiety
precision            0.545761
recall               0.544058
f1                   0.544908
roc_auc              0.784782
average_precision    0.582128
matthews             0.408153
spearman             0.416057
dtype: float64


n_estimators,95
_runtime,317
_timestamp,1629260321
_step,19
val precision,0.53734
val recall,0.52487
val f1,0.53103
val roc_auc,0.77676
val average_precision,0.56938
val matthews,0.3946
val spearman,0.40229


...

(52568, 1000) (52568,)
(2710, 1000) (2710,)


/data-imperial/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=25448     1.0
157416    1.0
171115    0.0
223106    1.0
1148      1.0
         ... 
239396    1.0
85563     1.0
84287     1.0
262507    1.0
319588    1.0
Name: 64_Yes, Length: 47311, dtype: float64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



64_Yes
precision            0.910504
recall               0.919389
f1                   0.914925
roc_auc              0.783043
average_precision    0.955571
matthews             0.325233
spearman             0.330020
dtype: float64


n_estimators,310
_runtime,148
_timestamp,1629263946
_step,62
val precision,0.9062
val recall,0.9159
val f1,0.91103
val roc_auc,0.74488
val average_precision,0.94613
val matthews,0.28576
val spearman,0.28449


n_estimators,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
val precision,▁
val recall,▁
val f1,▁
val roc_auc,▁
val average_precision,▁
val matthews,▁
val spearman,▁


## Current

In [7]:
config={"min_estimators":5,
"max_estimators":5000,
"max_depth":5,
"class_weight":"balanced",
"step_size":5,
"patience":3}

## list of labels to build model for
for label in tqdm([
    "18_anxiety", 
    "19_desire_x",
    "19_intent_x",
    "19_capability_x",
    "19_timeframe_x",
#     "69_13 or younger",
#     "69_14-24",
#     "69_24 or younger",
    "18_substance", 
    "18_depressed", 
    "18_self_harm",
#     "18_suicide",
    "64_Yes",
# #     "65>1 (slightly helpful)",
# #     "65>2",
# #     "65>3",
# #     "65>4",
# #     "241_I see a therapist or doctor in person",
#     "74_Heterosexual or Straight",
#     "75_White",
#     "73_Male",
# #     "33_good"
]):

    wandb.init(project="tfidf_baseline",reinit=True,
               config={
                   **config,
                "label":label
               },tags=['tfidf'])
    
    min_estimators = wandb.config.min_estimators
    max_estimators = wandb.config.max_estimators
    max_depth = wandb.config.max_depth
    class_weight = wandb.config.class_weight
    step_size = wandb.config.step_size
    patience = wandb.config.patience

    X, y = dropna_both(ds_train, l_train[label].astype('float'))
    X_test, y_test = dropna_both(ds_test, l_test[label].astype('float'))

    print(X.shape, y.shape)
    print(X_test.shape, y_test.shape)

    X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.1)

    u = np.unique(y_train)
    class_weight = {i:j for i,j in zip(u,compute_class_weight('balanced',u, y_train))}
    
    classifier = RandomForestClassifier(
        n_estimators = min_estimators,
        max_depth = max_depth,
        class_weight = class_weight,
        warm_start=True,
        n_jobs=2)
    
    fit_with_cv(X_train,y_train, X_val, y_val,
                model=classifier, 
                min_estimators = min_estimators, 
                max_estimators = max_estimators, 
                step_size = step_size, 
                patience = patience, 
                log_wandb=True)

    wandb.log({
        **prepend_keys(evaluate(classifier, X_train, X_val, y_train, y_val), "val "),
        **prepend_keys(evaluate(classifier, X_train, X_test, y_train, y_test), "test ")
    })
    print(label)
    print(pd.Series(evaluate(classifier, X_train, X_test, y_train, y_test)))
    wandb.join()

#     with open(f"models/rf/{label}_model.pickle", 'wb') as f:
#         pickle.dump(classifier, f)       
        

wandb: Currently logged in as: kiatann (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


(341955, 1000) (341955,)
(17997, 1000) (17997,)


/data-imperial/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=90465     0.0
74964     0.0
262405    0.0
297944    1.0
234886    0.0
         ... 
99883     0.0
272188    1.0
283125    1.0
125815    0.0
336372    0.0
Name: 18_anxiety, Length: 307759, dtype: float64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



18_anxiety
precision            0.544933
recall               0.547419
f1                   0.546173
roc_auc              0.786512
average_precision    0.600602
matthews             0.409117
spearman             0.418585
dtype: float64


n_estimators,100
_runtime,325
_timestamp,1629214743
_step,20
val precision,0.53825
val recall,0.53605
val f1,0.53715
val roc_auc,0.78306
val average_precision,0.5866
val matthews,0.40025
val spearman,0.41185


...

(52568, 1000) (52568,)
(2710, 1000) (2710,)


/data-imperial/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0. 1.], y=30423     1.0
168278    0.0
281446    1.0
7797      1.0
176774    1.0
         ... 
274848    0.0
201327    1.0
234718    0.0
58675     1.0
313178    0.0
Name: 64_Yes, Length: 47311, dtype: float64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "



64_Yes
precision            0.901879
recall               0.916419
f1                   0.909091
roc_auc              0.745061
average_precision    0.943469
matthews             0.263274
spearman             0.285734
dtype: float64


n_estimators,125
_runtime,63
_timestamp,1629217563
_step,25
val precision,0.89839
val recall,0.91493
val f1,0.90658
val roc_auc,0.73496
val average_precision,0.94078
val matthews,0.2502
val spearman,0.27586


n_estimators,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇███
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
val precision,▁
val recall,▁
val f1,▁
val roc_auc,▁
val average_precision,▁
val matthews,▁
val spearman,▁
